In [14]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset, random_split, Subset
from sklearn.metrics import average_precision_score
from sklearn.metrics import f1_score, precision_score, recall_score, average_precision_score
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.feature_selection import SelectKBest, f_classif

print("imports successful!")

imports successful!


In [3]:
x_train = torch.load("/scratch/gd2574/AudioSet-classification/Data/train/train_rep.pt")
y_train = torch.load("/scratch/gd2574/AudioSet-classification/Data/train/labels.pt")
x_test = torch.load("/scratch/gd2574/AudioSet-classification/Data/test/test_rep.pt")
y_test = torch.load("/scratch/gd2574/AudioSet-classification/Data/test/labels.pt")

/state/partition1/job-54426358/ipykernel_3159101/3713021574.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  x_train = torch.load("/scratch/gd2574/AudioSet-classification

# Data loaders

In [4]:
def create_stratified_split(x_data, y_data, random_state=42):
    mskf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)
    indices = np.arange(len(y_data))
    for train_idx, val_idx in mskf.split(indices, y_data):
        return train_idx, val_idx

# Helper functions

In [5]:
def calculate_map(y_true, y_pred):
    n_classes = y_true.shape[1]
    average_precisions = []
    for i in range(n_classes):
        y_true_class = y_true[:, i]
        y_pred_class = y_pred[:, i]
        ap = average_precision_score(y_true_class, y_pred_class)
        average_precisions.append(ap)
    average_precisions_sorted = sorted(average_precisions, reverse=True)
    # print("\nTop 5 class-wise Average Precisions:")
    # for i, ap in enumerate(average_precisions_sorted[:5]):
    #     print(f"Class {i+1}: {ap:.4f}")
    
    return np.mean(average_precisions)

# Train model

In [6]:
def train_mlp_model(model,optimizer,criterion, train_loader, val_loader,transform_type='mean', num_epochs=10):
    print(f"\nTraining MLP with {transform_type} transformation")
    model = model.to(device)
    
    best_map = 0
    
    for epoch in range(num_epochs):
        # Training 
        model.train()
        train_loss = 0
        for batch_x, batch_y in train_loader:
            batch_x = batch_x.float().to(device)
            batch_y = batch_y.float().to(device)
            
            optimizer.zero_grad()
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        # Validation 
        model.eval()
        val_predictions = []
        val_targets = []
        val_loss = 0
        
        with torch.no_grad():
            for batch_x, batch_y in val_loader:
                batch_x = batch_x.float().to(device)
                batch_y = batch_y.float().to(device)
                
                outputs = model(batch_x)
                loss = criterion(outputs, batch_y)
                val_loss += loss.item()
                val_predictions.extend(outputs.cpu().numpy())
                val_targets.extend(batch_y.cpu().numpy())
        
        val_predictions = np.array(val_predictions)
        val_targets = np.array(val_targets)
        val_map = calculate_map(val_targets, val_predictions)
        val_f1 = f1_score(val_targets, (val_predictions > 0.5).astype(float), average='micro')
        
        print(f'\nEpoch {epoch+1}/{num_epochs}:')
        print(f'Training Loss: {train_loss/len(train_loader):.4f}')
        print(f'Validation Loss: {val_loss/len(val_loader):.4f}')
        print(f'Validation MAP: {val_map:.4f}')
        print(f'Validation F1-Score: {val_f1:.4f}')
        
        if val_map > best_map:
            best_map = val_map
            torch.save(model.state_dict(), f'best_model_mlp_{transform_type}.pth')
            print("New best model saved!")
        
        print('-' * 50)
    
    return model, best_map

# Test model

In [7]:
def test_mlp_model(model, transform_type):
    print(f"\nTesting model with {transform_type} transformation")
    
    test_dataset = TensorDataset(x_test.float(), y_test.float())  
    test_loader = DataLoader(
        test_dataset, 
        batch_size=32, 
        shuffle=False,
        pin_memory=True if torch.cuda.is_available() else False
    )
    
    model_path = f'best_model_mlp_{transform_type}.pth'
    try:
        model.load_state_dict(torch.load(model_path))
        print(f"Successfully loaded model from {model_path}")
    except FileNotFoundError:
        print(f"Error: Could not find model file {model_path}")
        return
    except Exception as e:
        print(f"Error loading model: {str(e)}")
        return
    
    # Move model to correct device
    model = model.to(device)
    model.eval()  
    
    test_predictions = []
    test_targets = []  
    
    with torch.no_grad():
        for batch_x, batch_y in test_loader: 
            batch_x = batch_x.float().to(device)
            batch_y = batch_y.float().to(device)
            
            outputs = model(batch_x)
            test_predictions.extend(outputs.cpu().numpy())
            test_targets.extend(batch_y.cpu().numpy())
                
    test_predictions = np.array(test_predictions)
    test_targets = np.array(test_targets) 
    
    test_map = calculate_map(test_targets, test_predictions)
    test_f1 = f1_score(test_targets, (test_predictions > 0.5).astype(float), average="micro")
    
    print(f"Test MAP: {test_map:.4f}")
    print(f"Test F1-Score: {test_f1:.4f}")
    print("-" * 50)
    
    return test_map, test_f1

print("Beginning model testing...")
test_results = {}
transforms = ['mean', 'max', 'append']

for transform in transforms:
    try:
        model = MLPWithTransform(transform_type=transform)
        test_map, test_f1 = test_model(model, transform)
        test_results[transform] = {
            'map': test_map,
            'f1': test_f1
        }
    except Exception as e:
        print(f"Error testing {transform} transform model:", str(e))

# Print final comparison
print("\nFinal Test Results:")
print("-" * 50)
for transform, metrics in test_results.items():
    print(f"{transform.capitalize()} Transform:")
    print(f"  MAP: {metrics['map']:.4f}")
    print(f"  F1:  {metrics['f1']:.4f}")

Beginning model testing...
Error testing mean transform model: name 'MLPWithTransform' is not defined
Error testing max transform model: name 'MLPWithTransform' is not defined
Error testing append transform model: name 'MLPWithTransform' is not defined

Final Test Results:
--------------------------------------------------


# MLP

In [15]:
class MLPWithTransform(nn.Module):
    def __init__(self, input_size=768, hidden_size=1024, num_classes=527, transform_type='mean'):
        super(MLPWithTransform, self).__init__()
        self.transform_type = transform_type
        
        self.layers = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_size),
            nn.Dropout(0.3),
            
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_size),
            nn.Dropout(0.3),
            
            nn.Linear(hidden_size, hidden_size // 2),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_size // 2),
            nn.Dropout(0.3),
            
            nn.Linear(hidden_size // 2, num_classes),
            nn.Sigmoid()
        )
    
    def transform_sequence(self, x):
        # x shape: (batch_size, seq_len, features)
        if self.transform_type == 'mean':
            return torch.mean(x, dim=1)  # Average across sequence length
        elif self.transform_type == 'max':
            return torch.max(x, dim=1)[0]  # Max across sequence length
        elif self.transform_type == 'append':
            # Flatten the input
            flattened_x = x.view(x.size(0), -1).numpy()
            selector = SelectKBest(f_classif, k=1000)
            selected_x = selector.fit_transform(flattened_x, y_train.numpy())
            selected_x = torch.tensor(selected_x)
            return selected_x #x.view(x.size(0), -1)
            # return x[:, 0, :]  # Take first timestep features
        else:
            raise ValueError(f"Unknown transform type: {self.transform_type}")
    
    def forward(self, x):
        x = self.transform_sequence(x)  # Transform the sequence
        return self.layers(x)

# lessgooo

In [16]:
batch_size = 32
learning_rate= 0.001
num_epochs = 2
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
x_train_float = x_train.float()
y_train_float = y_train.float()
full_dataset = TensorDataset(x_train_float, y_train_float)
total_size = len(full_dataset)

# USING STRATIFIED SPLITS
train_indices, val_indices = create_stratified_split(x_train_float, y_train_float.numpy())

train_dataset = Subset(full_dataset, train_indices)
val_dataset = Subset(full_dataset, val_indices)
train_loader = DataLoader(train_dataset,batch_size=32,  shuffle=True, pin_memory=True if torch.cuda.is_available() else False)
val_loader = DataLoader(val_dataset, batch_size=32,pin_memory=True if torch.cuda.is_available() else False)
print(f"Total samples: {len(full_dataset)}")
print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")

criterion = nn.BCELoss()

transforms = ['mean', 'max', 'append']
results = {}

Total samples: 20550
Training samples: 16440
Validation samples: 4110


In [17]:
for transform in transforms:
    try:
        model = MLPWithTransform(transform_type=transform)
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        model, best_map = train_mlp_model(model,optimizer,criterion,train_loader, val_loader, transform_type=transform, num_epochs=num_epochs)
        results[transform] = {'model': model, 'best_map': best_map}
    except Exception as e:
        print(f"Error training with {transform} transform:", str(e))

print("\nFinal Results:")
for transform, result in results.items():
    print(f"{transform} transform - Best MAP: {result['best_map']:.4f}")



Training MLP with mean transformation

Epoch 1/2:
Training Loss: 0.1396
Validation Loss: 0.0206
Validation MAP: 0.1016
Validation F1-Score: 0.2114
New best model saved!
--------------------------------------------------

Epoch 2/2:
Training Loss: 0.0201
Validation Loss: 0.0167
Validation MAP: 0.2181
Validation F1-Score: 0.3129
New best model saved!
--------------------------------------------------

Training MLP with max transformation

Epoch 1/2:
Training Loss: 0.1376
Validation Loss: 0.0230
Validation MAP: 0.0267
Validation F1-Score: 0.1849
New best model saved!
--------------------------------------------------

Epoch 2/2:
Training Loss: 0.0228
Validation Loss: 0.0203
Validation MAP: 0.0811
Validation F1-Score: 0.2297
New best model saved!
--------------------------------------------------

Training MLP with append transformation
Error training with append transform: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

Final R

In [12]:
for transform in transforms:
    test_mlp_model(model,transform_type=transform)


Testing model with mean transformation
Successfully loaded model from best_model_mlp_mean.pth


/state/partition1/job-54426358/ipykernel_3159101/3848944109.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


Test MAP: 0.0372
Test F1-Score: 0.1516
--------------------------------------------------

Testing model with max transformation
Successfully loaded model from best_model_mlp_max.pth


/state/partition1/job-54426358/ipykernel_3159101/3848944109.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


Test MAP: 0.0085
Test F1-Score: 0.1570
--------------------------------------------------

Testing model with append transformation
Successfully loaded model from best_model_mlp_append.pth


/state/partition1/job-54426358/ipykernel_3159101/3848944109.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


Test MAP: 0.2752
Test F1-Score: 0.3890
--------------------------------------------------
